In [6]:
import re
from collections import namedtuple

In [28]:
from typing import NamedTuple

In [83]:
import operator as op
from functools import partial

class Amounts(NamedTuple):
    ore: int = 0
    clay: int = 0
    obsidian: int = 0
    geode: int = 0
    
    def __add__(self, other: Amounts): return Amounts(*map(op.add,self,other))
    def __sub__(self, other: Amounts): return Amounts(*map(op.sub,self,other))
    def __mul__(self, other):
        if isinstance(other, Amounts):
            return Amounts(*map(op.mul,self,other))
        else:
            return Amounts(*map(partial(op.mul,other),self))
            


In [84]:
BP = namedtuple("Blueprint", "ore,clay,obsidian,geode")

In [85]:
def parse(fname):
    for line in open(fname):
        if line.strip():
            _,*ns= re.findall('[0-9]+', line)
            ns = map(int,ns)
            yield BP(ore=Amounts(ore=next(ns)), 
                               clay=Amounts(ore=next(ns)),
                               obsidian=Amounts(ore=next(ns),clay=next(ns)),
                                geode=Amounts(ore=next(ns),obsidian=next(ns)))
            
bps = list(parse("test"))
bps

[Blueprint(ore=Amounts(ore=4, clay=0, obsidian=0, geode=0), clay=Amounts(ore=2, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=3, clay=14, obsidian=0, geode=0), geode=Amounts(ore=2, clay=0, obsidian=7, geode=0)),
 Blueprint(ore=Amounts(ore=2, clay=0, obsidian=0, geode=0), clay=Amounts(ore=3, clay=0, obsidian=0, geode=0), obsidian=Amounts(ore=3, clay=8, obsidian=0, geode=0), geode=Amounts(ore=3, clay=0, obsidian=12, geode=0))]

In [125]:
def evaluate(moves: list, bp: BP, t=24):
    minerals = Amounts()
    robots = Amounts(ore=1)
    breakpoint()
    while t > 0:
        #build robots
        move = moves.pop(0) if len(moves) > 0 else None
        if move:
            for i in range(4):
                minerals = minerals - bp[i]*move[i]
            
            if any(m < 0 for m in minerals):
                raise ValueError(minerals)
            
            new_robots = robots + move
            
        #collect
        minerals = minerals + robots
        
        #complete robots
        if move:
            robots = new_robots
        
        print(24-t+1,minerals)
        t-=1
    return minerals

moves=[
    None,None,Amounts(clay=1),
    None,Amounts(clay=1),None,
    Amounts(clay=1),None,None,
    None,Amounts(obsidian=1),Amounts(clay=1),
    None,None,Amounts(obsidian=1),
    None,None,Amounts(geode=1),
    None,None,Amounts(geode=1),
    None,None,None
]

evaluate(moves,bps[0])

1 Amounts(ore=1, clay=0, obsidian=0, geode=0)
2 Amounts(ore=2, clay=0, obsidian=0, geode=0)
3 Amounts(ore=1, clay=0, obsidian=0, geode=0)
4 Amounts(ore=2, clay=1, obsidian=0, geode=0)
5 Amounts(ore=1, clay=2, obsidian=0, geode=0)
6 Amounts(ore=2, clay=4, obsidian=0, geode=0)
7 Amounts(ore=1, clay=6, obsidian=0, geode=0)
8 Amounts(ore=2, clay=9, obsidian=0, geode=0)
9 Amounts(ore=3, clay=12, obsidian=0, geode=0)
10 Amounts(ore=4, clay=15, obsidian=0, geode=0)
11 Amounts(ore=2, clay=4, obsidian=0, geode=0)
12 Amounts(ore=1, clay=7, obsidian=1, geode=0)
13 Amounts(ore=2, clay=11, obsidian=2, geode=0)
14 Amounts(ore=3, clay=15, obsidian=3, geode=0)
15 Amounts(ore=1, clay=5, obsidian=4, geode=0)
16 Amounts(ore=2, clay=9, obsidian=6, geode=0)
17 Amounts(ore=3, clay=13, obsidian=8, geode=0)
18 Amounts(ore=2, clay=17, obsidian=3, geode=0)
19 Amounts(ore=3, clay=21, obsidian=5, geode=1)
20 Amounts(ore=4, clay=25, obsidian=7, geode=2)
21 Amounts(ore=3, clay=29, obsidian=2, geode=3)
22 Amounts(or

Amounts(ore=6, clay=41, obsidian=8, geode=9)

In [179]:
def minerals_spent_building_robots(robots, bp):
    minerals = Amounts()
    for i in range(4):
        minerals = minerals + bp[i]*robots[i]   
    return minerals

def can_build_robots(robots, minerals, bp):
    minerals = minerals - minerals_spent_building_robots(robots,bp)
    return all(m >= 0 for m in minerals)


In [184]:
can_build_robots(Amounts(1,0),Amounts(4),bps[0])

True

In [185]:

def valid_moves(robots, minerals, bp):    
    # just moves of one robot at a time for now
    for i in range(4):
        robot = [0,0,0,0]
        robot[i] = 1
        if can_build_robots(Amounts(*robot), minerals,bp):
            yield robot

list(valid_moves(Amounts(), Amounts(2),bps[0]))

[[0, 1, 0, 0]]

In [264]:
from collections import deque

def solve(bp: BP, t=24):

    best = 0
    # state is a tuple of (robots, minerals, time_left)
    todo = deque([(Amounts(ore=1), Amounts(), t)])
    visited = {}    
    
    it = 0
    while todo:
        it += 1
        robots, minerals, time_left = state = todo.pop()
        
        if (robots, minerals) in visited and visited[(robots,minerals)] >= time_left:
            continue
        else:
            visited[(robots, minerals)] = time_left
            
        best = max(best, minerals.geode)
        
        if (it % 100_000 == 0): print(it, len(todo), len(visited), time_left, best)

        if time_left == 0:
            continue
 
        for move in valid_moves(robots, minerals, bp):
            new_state = (
                robots + move,
                minerals - minerals_spent_building_robots(move, bp) + robots,
                time_left - 1
            )
            todo.append(new_state)
            
        
        # also add the do nothing move
        todo.append((robots, minerals + robots, time_left - 1))
        
    return best

In [ ]:
solve(bps[1])

200000 33 89431 0 0
300000 27 130161 0 0
400000 39 165522 0 0
500000 31 199524 0 0
700000 32 272774 0 0
800000 33 302727 0 0
900000 30 335672 0 0
1100000 25 393857 0 0
1200000 32 428676 0 0
1300000 29 467010 0 0
1400000 35 500162 0 0
1500000 26 528282 1 1
1600000 33 558241 0 1
1700000 27 584672 2 1
2100000 24 702552 3 1
2200000 25 727809 2 1
2400000 40 787671 0 1
2700000 33 883992 0 1
2800000 38 928942 0 1
3000000 36 989277 1 1
3300000 22 1077348 0 1
3400000 35 1099731 1 1
3500000 24 1123790 2 1
3600000 27 1148874 0 1
4100000 43 1288157 0 1
4200000 23 1320292 4 1
4500000 18 1399728 5 2
4700000 23 1453999 2 2
4800000 23 1482125 5 2
5100000 21 1568570 0 2
5200000 25 1604239 0 2
5300000 28 1636894 0 2
5400000 24 1665021 1 2
5500000 24 1688975 3 2
5700000 42 1746364 0 2
5800000 28 1788884 0 2
6000000 27 1869358 1 2
6200000 30 1929844 0 2
6300000 29 1968981 0 2
6600000 25 2052110 0 2
6700000 20 2083466 2 2
6800000 20 2105692 1 2
7100000 27 2178967 0 2
7300000 34 2225144 0 2
7500000 43 22804